this is sample code from Microsoft OpenAI Chat Sandbox

In [15]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# Print expected environment variables and their actual values
print("Expected Environment Variables and Current Values:")
print("=" * 60)

endpoint_env = os.environ.get('AZURE_OPENAI_ENDPOINT', 'NOT SET')
# Use the endpoint exactly as provided by the user (support either with or without '/openai' path)
if endpoint_env != 'NOT SET':
    endpoint = endpoint_env.rstrip('/')
else:
    endpoint = 'NOT SET'

deployment_name = os.environ.get('AZURE_OPENAI_DEPLOYMENT', 'NOT SET')
api_key = os.environ.get('AZURE_OPENAI_API_KEY', 'NOT SET')
api_version = os.environ.get('AZURE_OPENAI_API_VERSION', os.environ.get('OPENAI_API_VERSION', '2024-02-01'))

# sanitize api_version in case the .env value contains extra quotes or comments
if isinstance(api_version, str):
    api_version = api_version.split()[0].strip("'\"")

print(f"AZURE_OPENAI_ENDPOINT: {endpoint}")
print(f"AZURE_OPENAI_DEPLOYMENT: {deployment_name}")
print(f"AZURE_OPENAI_API_KEY: {'***' + api_key[-4:] if api_key != 'NOT SET' and len(api_key) > 4 else api_key}")
print(f"API_VERSION: {api_version}")
print("=" * 60)
print()

# Validate required variables are set
if endpoint == 'NOT SET' or deployment_name == 'NOT SET' or api_key == 'NOT SET':
    raise ValueError("Missing required environment variables. Please check your .env file.")

client = OpenAI(
    base_url=endpoint,
    api_key=api_key
)

prompt = "Oh say can you see"
try:
    completion = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
    print(f"Prompt: {prompt}")
    print(f"Response: {completion.choices[0].message.content}")
except Exception as e:
    # If a 404 occurs, retry once with the '/openai' suffix if the original endpoint didn't include it.
    err_str = str(e)
    if ('404' in err_str or getattr(e, 'status_code', None) == 404) and endpoint_env != 'NOT SET' and '/openai' not in endpoint_env:
        print("Received 404 error. Retrying with endpoint including '/openai' path suffix...")
        endpoint_with_openai = endpoint + '/openai'
        client = OpenAI(base_url=endpoint_with_openai, api_key=api_key)
        try:
            completion = client.chat.completions.create(
                model=deployment_name,
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
            )
            print(f"Prompt: {prompt}")
            print(f"Response: {completion.choices[0].message.content}")
        except Exception as e2:
            print("Retry failed:", e2)
            raise
    else:
        print("Request failed:", e)
        print("Check that AZURE_OPENAI_ENDPOINT is correct (include or omit '/openai' as needed) and AZURE_OPENAI_DEPLOYMENT matches your deployed model name.")
        raise

Expected Environment Variables and Current Values:
AZURE_OPENAI_ENDPOINT: https://mwool-mj66edpx-eastus2.openai.azure.com/openai/v1
AZURE_OPENAI_DEPLOYMENT: gpt-5.2-chat
AZURE_OPENAI_API_KEY: ***qbvK
API_VERSION: 2024-02-01

Prompt: Oh say can you see
Response: By the dawn’s early light  
What so proudly we hail’d at the twilight’s last gleaming,  
Whose broad stripes and bright stars through the perilous fight  
O’er the ramparts we watched, were so gallantly streaming?  
And the rockets’ red glare, the bombs bursting in air,  
Gave proof through the night that our flag was still there;  
Oh say does that star-spangled banner yet wave  
O’er the land of the free and the home of the brave?
